<a href="https://colab.research.google.com/github/redish0317/cp1/blob/main/cp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
from datetime import *
warnings.simplefilter(action='ignore', category=FutureWarning)

In [190]:
def crawl_id():
    url = 'http://www.wanted.jobs/api/v4/jobs?'

    params ={1656232918453:'',    #사용자번호?
    'country': 'all',
    'tag_type_ids': 873,    #직무 카테고리 고유 id
    'job_sort': 'job.latest_order',    #최신순 정렬
    'locations': all,
    'years': -1,    #경력 이상
    'years': -1,    #경력 이하    경력상관없이 검색하려면 -1
    'limit': 10,
    'offset': 10}

    r = requests.get(url,
                     params = params)
    r = r.json()
    
    df = pd.json_normalize(r['data'])

    return df

def crawl_job(id_list):
    df = pd.DataFrame()
    for id in id_list:
        url = f'https://www.wanted.jobs/api/v4/jobs/{id}?1656259528432'
        r = requests.get(url)
        r = r.json()['job']
        df = pd.concat([df,pd.json_normalize(r)],axis=0)
    return df

def engineering(df):
    drop_col = ['status','is_like','is_bookmark','compare_country','matching_score','reward.formatted_total','reward.formatted_recommender','reward.formatted_recommendee','title_img.origin','title_img.thumb',
            'logo_img.origin','logo_img.thumb']
    df = df.drop(drop_col,axis=1)
    return df

In [191]:
df_id = crawl_id()

df_job = crawl_job(df_id['id'])

df = pd.concat([df_id,df_job],axis=0)

df = engineering(df)

df.head()

,due_time,like_count,id,position,score,category_tags,company.id,company.industry_name,company.application_response_stats.avg_rate,company.application_response_stats.level,...,address.geo_location.viewport.northeast.lng,address.geo_location.viewport.southwest.lat,address.geo_location.viewport.southwest.lng,address.geo_location.bounds,address.country_code,detail.requirements,detail.main_tasks,detail.intro,detail.benefits,detail.preferred_points
0,None,5,70904,Web Front-end Developer 웹 프론트엔드 개발자,None,"[{'parent_id': 518, 'id': 669}, {'parent_id': ...",155,"IT, Media",86.7,normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,51,78227,"웹 프론트엔드 개발자(JavaScript, React)(병역특례/산업기능요원/보충역...",None,"[{'parent_id': 518, 'id': 660}, {'parent_id': ...",15353,"IT, Media",96.0,very_high,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-07-08,0,118985,웹 서비스 개발/운영,None,"[{'parent_id': 518, 'id': 872}, {'parent_id': ...",17615,"IT, Media",98.1,very_high,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-07-04,0,99826,ADEX 개발본부 개발자 (웹 서비스 및 공통 플랫폼 개발),None,"[{'parent_id': 518, 'id': 660}, {'parent_id': ...",604,"IT, Media",96.3,very_high,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-07-08,0,118984,e커머스 개발/운영,None,"[{'parent_id': 518, 'id': 669}, {'parent_id': ...",17615,"IT, Media",98.1,very_high,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#폐기

In [204]:
def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup
url = 'https://www.wanted.jobs/wd/109904'

soup = str(make_soup(url))

url_list = ['https://www.wanted.jobs/wd/'+ str(i) for i in df['id']]

#서론
Introduction = soup[soup.find('"jd":"') + 6 : soup.find('\\n\\n\\n주요업무\\n')]

#주요업무, 책임
responsibilities = soup[soup.find('\\n\\n\\n주요업무\\n') + 12 : soup.find('\\n\\n자격요건\\n')]

#요구자격
basic_qualifications = soup[soup.find('\\n\\n자격요건\\n') + 12 : soup.find('\\n\\n우대사항\\n')]

#우대자격
preferred_qualifications = soup[soup.find('\\n\\n우대사항\\n') + 10 : soup.find('\\n\\n혜택 및 복지\\n###')]

#복지
company_benefits = soup[soup.find('\\n\\n혜택 및 복지\\n###') + 16 : soup.find('","company_name":"')]

pd.DataFrame({'Introduction':Introduction,
              'responsibilities':responsibilities,
              'basic_qualifications':basic_qualifications,
              'preferred_qualifications':preferred_qualifications,
              'company_benefits':company_benefits},index=[0])

,Introduction,responsibilities,basic_qualifications,preferred_qualifications,company_benefits
0,"빌리오(BIllyo)\n#크리에이터의 고민 해결!\n콘텐츠 제작 커뮤니티, ‘빌리오...",[주요 업무]\n- 빌리오(Billyo) 모바일 서비스를 위한 API 개발 및 유지...,Python 기반 Restful API 설계 및 구현 경험이 3년 이상 있으시거나 ...,- O2O 플랫폼 비즈니스에 대한 흥미와 관심이 있는 분\n- 크리에이터 시장에 관...,"이런 게 좋아요!\n빌리오 팀원들은 함께하는 동료를 소중히 여기며, 함께 성장하면..."
